In [1]:
import pandas as pd
import psycopg2
import sqlalchemy
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [2]:
from sqlalchemy import create_engine

# Postgres username, password, and database name
POSTGRES_ADDRESS = 'mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'lhl_student'
POSTGRES_PASSWORD = 'lhl_student'
POSTGRES_DBNAME = 'mid_term_project'

# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME, \
                                                                                        password=POSTGRES_PASSWORD, \
                                                                                        ipaddress=POSTGRES_ADDRESS, \
                                                                                        port=POSTGRES_PORT, \
                                                                                        dbname=POSTGRES_DBNAME))

# Create the connection
con = create_engine(postgres_str)

### Load data from flights dataset for analysis and data preparation

In [3]:
# Select random data to get a representative sample (taking more positive samples so data stays balanced)
sample_pos = pd.read_sql_query('SELECT * FROM flights WHERE arr_delay > 0 ORDER BY random() LIMIT 55000;', con)

In [4]:
# Select random data to get a representative sample
sample_neg = pd.read_sql_query('SELECT * FROM flights WHERE arr_delay < 0 ORDER BY random() LIMIT 50000;', con)

In [5]:
sample = pd.concat([sample_pos, sample_neg], ignore_index=True)

In [6]:
sample.shape

(105000, 42)

In [7]:
sample.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2018-06-18,DL,DL,DL,2653,DL,N321US,2653,14492,RDU,"Raleigh/Durham, NC",10397,ATL,"Atlanta, GA",730,723.0,-7.0,38.0,801.0,853.0,6.0,856,859.0,3.0,0.0,None,0.0,N,86.0,96.0,52.0,1.0,356.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
1,2019-11-30,DL,DL_CODESHARE,DL,5211,9E,N8980A,5211,13487,MSP,"Minneapolis, MN",11203,CWA,"Mosinee, WI",1019,1014.0,-5.0,48.0,1102.0,1140.0,6.0,1117,1146.0,29.0,0.0,None,0.0,N,58.0,92.0,38.0,1.0,175.0,0,0,29,0,0,None,NaN,NaN,None
2,2018-10-23,AA,AA_CODESHARE,AA,4715,YX,N124HQ,4715,14576,ROC,"Rochester, NY",11057,CLT,"Charlotte, NC",1850,1842.0,-8.0,13.0,1855.0,2028.0,28.0,2055,2056.0,1.0,0.0,None,0.0,N,125.0,134.0,93.0,1.0,573.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
3,2019-04-29,AA,AA,AA,2807,AA,N957AN,2807,13303,MIA,"Miami, FL",15304,TPA,"Tampa, FL",1335,1356.0,21.0,22.0,1418.0,1458.0,10.0,1438,1508.0,30.0,0.0,None,0.0,N,63.0,72.0,40.0,1.0,204.0,21,0,9,0,0,None,NaN,NaN,None
4,2018-07-17,UA,UA_CODESHARE,UA,5698,OO,N925EV,5698,13930,ORD,"Chicago, IL",14696,SBN,"South Bend, IN",1944,2035.0,51.0,19.0,2054.0,2214.0,5.0,2135,2219.0,44.0,0.0,None,0.0,N,51.0,44.0,20.0,1.0,84.0,0,0,0,0,44,None,NaN,NaN,None


In [8]:
# save data to .csv for later use
sample.to_csv('sample.csv')

### Load data from fuel consumption dataset for analysis

In [9]:
fuel = pd.read_sql_query('SELECT * from fuel_comsumption LIMIT 5;', con)
fuel

,month,airline_id,unique_carrier,carrier,carrier_name,carrier_group_new,sdomt_gallons,satl_gallons,spac_gallons,slat_gallons,sint_gallons,ts_gallons,tdomt_gallons,tint_gallons,total_gallons,sdomt_cost,satl_cost,spac_cost,slat_cost,sint_cost,ts_cost,tdomt_cost,tint_cost,total_cost,year
0,1,NaN,None,0JQ,None,1,0.0,0.0,0.0,0.0,0.0,0.0,3000.0,0.0,3000.0,0.0,0.0,0.0,0.0,0.0,0.0,5100.0,0.0,5100.0,2016
1,1,21352.0,0WQ,0WQ,Avjet Corporation,1,0.0,0.0,0.0,0.0,0.0,0.0,163052.0,47060.0,210112.0,0.0,0.0,0.0,0.0,0.0,0.0,396216.0,140239.0,536455.0,2016
2,1,21645.0,23Q,23Q,Songbird Airways Inc.,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016
3,1,21652.0,27Q,27Q,"Jet Aviation Flight Services, Inc.",1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016
4,1,20408.0,5V,5V,Tatonduk Outfitters Limited d/b/a Everts Air A...,1,260848.0,0.0,0.0,0.0,0.0,260848.0,284362.0,0.0,284362.0,522405.0,0.0,0.0,0.0,0.0,522405.0,569497.0,0.0,569497.0,2016


### Load data from passengers dataset for analysis

In [10]:
passengers = pd.read_sql_query('SELECT * from passengers LIMIT 5;', con)
passengers

,departures_scheduled,departures_performed,payload,seats,passengers,freight,mail,distance,ramp_to_ramp,air_time,unique_carrier,airline_id,unique_carrier_name,region,carrier,carrier_name,carrier_group,carrier_group_new,origin_airport_id,origin_city_market_id,origin,origin_city_name,origin_country,origin_country_name,dest_airport_id,dest_city_market_id,dest,dest_city_name,dest_country,dest_country_name,aircraft_group,aircraft_type,aircraft_config,year,month,distance_group,class,data_source
0,30.0,30.0,886500.0,3840.0,3549.0,2326.0,27870.0,1597.0,6332.0,5532.0,UA,19977,United Air Lines Inc.,D,UA,United Air Lines Inc.,3,3,12266,31453,IAH,"Houston, TX",US,United States,10721,30721,BOS,"Boston, MA",US,United States,6,698,1,2015,3,4,F,DU
1,30.0,30.0,752400.0,3540.0,2492.0,0.0,0.0,934.0,4816.0,4190.0,UA,19977,United Air Lines Inc.,D,UA,United Air Lines Inc.,3,3,12266,31453,IAH,"Houston, TX",US,United States,11503,31503,EGE,"Eagle, CO",US,United States,6,612,1,2015,3,2,F,DU
2,30.0,30.0,1062233.0,5034.0,4348.0,0.0,4426.0,1824.0,6862.0,6227.0,UA,19977,United Air Lines Inc.,D,UA,United Air Lines Inc.,3,3,12889,32211,LAS,"Las Vegas, NV",US,United States,11042,30647,CLE,"Cleveland, OH",US,United States,6,634,1,2015,3,4,F,DU
3,30.0,28.0,827400.0,3584.0,2599.0,290.0,0.0,594.0,3519.0,2804.0,UA,19977,United Air Lines Inc.,D,UA,United Air Lines Inc.,3,3,13230,32070,MDT,"Harrisburg, PA",US,United States,13930,30977,ORD,"Chicago, IL",US,United States,6,698,1,2015,3,2,F,DU
4,30.0,30.0,886500.0,3840.0,3338.0,630.0,635.0,1062.0,4939.0,4430.0,UA,19977,United Air Lines Inc.,D,UA,United Air Lines Inc.,3,3,13495,33495,MSY,"New Orleans, LA",US,United States,11292,30325,DEN,"Denver, CO",US,United States,6,698,1,2015,3,3,F,DU


### Load data from flights_test dataset for final predictions

In [11]:
flights_test = pd.read_sql_query('SELECT * FROM flights_test LIMIT 155000;', con)

In [12]:
flights_test.tail(5)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
154995,2020-01-08,AA,AA,AA,1748,AA,N174US,1748,15304,TPA,"Tampa, FL",11057,CLT,"Charlotte, NC",1726,1910,N,104,1,507
154996,2020-01-08,AA,AA,AA,1749,AA,N108UW,1749,11057,CLT,"Charlotte, NC",14122,PIT,"Pittsburgh, PA",2019,2145,N,86,1,366
154997,2020-01-08,AA,AA,AA,1751,AA,N837AW,1751,13198,MCI,"Kansas City, MO",13930,ORD,"Chicago, IL",1409,1548,N,99,1,403
154998,2020-01-08,AA,AA,AA,1751,AA,N837AW,1751,13930,ORD,"Chicago, IL",13198,MCI,"Kansas City, MO",1155,1329,N,94,1,403
154999,2020-01-08,AA,AA,AA,1754,AA,N109UW,1754,11057,CLT,"Charlotte, NC",13230,MDT,"Harrisburg, PA",1634,1806,N,92,1,413


In [13]:
flights_test = flights_test[flights_test['fl_date']!='2020-01-08']
flights_test.tail(5)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
150618,2020-01-07,DL,DL_CODESHARE,DL,4813,9E,N605LR,4813,11433,DTW,"Detroit, MI",12478,JFK,"New York, NY",1755,1952,N,117,1,509
150619,2020-01-07,DL,DL_CODESHARE,DL,4814,9E,N919XJ,4814,11996,GSP,"Greer, SC",12953,LGA,"New York, NY",600,759,N,119,1,610
150620,2020-01-07,DL,DL_CODESHARE,DL,4815,9E,N936XJ,4815,10397,ATL,"Atlanta, GA",15919,XNA,"Fayetteville, AR",1715,1816,N,121,1,589
150621,2020-01-07,DL,DL_CODESHARE,DL,4815,9E,N936XJ,4815,15919,XNA,"Fayetteville, AR",10397,ATL,"Atlanta, GA",1851,2145,N,114,1,589
150622,2020-01-07,DL,DL_CODESHARE,DL,4816,9E,N917XJ,4816,14321,PWM,"Portland, ME",12953,LGA,"New York, NY",600,741,N,101,1,269


In [14]:
# save data to .csv for later use
flights_test.to_csv('flights_test.csv')